In [ ]:
#| default_exp core

# core

> Fill in a module description here

In [ ]:
#| export
from fastcore.script import *
from pathlib import Path
import polars as pl
from gensim.models import Word2Vec

In [ ]:
#| export
def get_docs(
    dir_comments: str, # directory containing parquet dataframes
    subreddit: str # subreddit to be processed
    ) -> list:
    dir_comments = Path(dir_comments)
    df = pl.read_parquet(dir_comments / subreddit / '*.parquet')
    docs_srs = df['body'].sample(1_000).str.split(' ')
    docs_list = docs_srs.to_list()
    return docs_list
    

# train model

In [ ]:
#| export
@call_parse
def train_model(
    subreddit: str,
    dir_comments: str,
    dir_models: str
    ):
    docs = get_docs(dir_comments, subreddit)
    model = Word2Vec(
        docs[:1_000_000],
        workers=8
    )
    dir_models = Path(dir_models)
    model.save((dir_models / f"{subreddit}.model").as_posix())

# toy console script

In [ ]:
#| export
@call_parse
def add_one(
    num: int # first number
    ) -> int: # result
    result = num + 1
    print(result)
    return result